In [33]:
import re
import numpy as np
import pandas as pd
import difflib
import transformers
import spacy
from transformers import BertTokenizerFast

In [51]:
feature_words = ["america","bank","health"]#? split commonly used words
commonly_used_words = ["america","bank","health","pharma","pharmaceutical","us","partners","united states","financial","companies","services","brands","air","airlines","international","asset","equity","fund","group","resourses","technologies","hotels","control","black","green","natural"]

In [35]:
df_labels = pd.read_csv("dslim/nasdaq_labeled_companies_holdings.csv",index_col=False,header=0)
df_all_extracted_companies = pd.read_csv("dslim/all_companies_nasdaq.csv",index_col=False,header=0)
df_news = pd.read_csv("dslim/nasdaq_labeled_news.csv",index_col=False,header=0)

In [36]:
nlp = spacy.load("en_core_web_sm")
COMMENT = ""

In [37]:
tokenizer = transformers.BertTokenizerFast.from_pretrained("dslim/bert-base-NER")

In [38]:
print(commonly_used_words[0])
print("brands" in commonly_used_words)

america
True


In [39]:
def is_valid_match(real_str, extracted_str):
    global COMMENT
    
    all_real_words_is_common = True
    all_extracted_words_is_common = True
    real_words = real_str.lower().replace("-","").split(" ")#for walmart and other
    extracted_words = extracted_str.lower().replace("-","").split(" ")
    for word in real_words:
        if(word not in commonly_used_words and len(word) > 3):
            all_real_words_is_common = False
    if(all_real_words_is_common):
        return False
    for word in extracted_words:
        if(word not in commonly_used_words and len(word) > 3):
            all_extracted_words_is_common = False
    if(all_extracted_words_is_common):
        return False
    
    for word in extracted_words:
        if(word not in real_words):
            return False
    COMMENT = "Vaild words match"
    return True

In [40]:
def is_also_proper(real_str, extracted_str,nlp):
    global COMMENT
    doc = nlp(real_str)
    for tok in doc: 
        if(not tok.text.isupper() and tok.pos_ == 'PROPN' and extracted_str.replace("-","").title().find(tok.text)==-1 and tok.text not in tokenizer.get_vocab() and len(tok.text)>3 and tok.text.lower not in commonly_used_words):
            return False
        if(tok.pos_ == 'PROPN' and extracted_str.replace("-","").title().find(tok.text)!=-1 and len(tok.text)>3):
            COMMENT = "Both companies have same proper name"
            return True
    return False

In [41]:
def is_also_person(real_str, extracted_str,nlp):
    global COMMENT
    doc = nlp(real_str)
    for ent in doc.ents: 
        if(ent.label_ == 'PERSON' and extracted_str.find(ent.text)!=-1 and len(ent.label > 2)):
            COMMENT = "Both companies have same person name"
            return True
    return False

In [42]:
def is_both_unique(real_str, extracted_str):#check if organisation name contains unique word
    global COMMENT
    real_words = real_str.replace("-","").split(" ")
    extracted_words = extracted_str.replace("-","").split(" ")
    if(len(real_words)==1 and len(extracted_str.replace("-","").split(" "))==1):#one subword - not valid
        return False
    for word in real_words:
        if(word in extracted_words and word not in tokenizer.get_vocab() and word.lower() not in commonly_used_words):
            COMMENT = "Both companies have unique word"
            return True
    words = real_str.split(" ")
    for word in words:
        if(word in extracted_words and word not in tokenizer.get_vocab() and word.lower() not in commonly_used_words):
            COMMENT = "Both companies have unique word"
            return True
    return False

In [43]:
def is_extracted_contain_real(real_str, extracted_str):#check if organisation name contains person name
    words = real_str.replace("-"," ").lower().split(" ")
    extracted_words = extracted_str.replace("-"," ").lower().split(" ")
    for word in words:
        if(word not in extracted_words):
            return False
    return True

In [44]:
def is_abbreviation(real_str, extracted_str):
    global COMMENT
    company = real_str.lower()
    extracted = extracted_str.replace("-"," ").lower().split(" ")
    extracted = [elem for elem in extracted if len(elem)!=0]
    if(len(company)!=len(extracted)): 
        extracted.append("group")
    if(len(company)!=len(extracted)):
        return False
    for idx in range(len(extracted)):
        if(company[idx]!=extracted[idx][0]):
            return False
    COMMENT = "Company is abbreviation"
    return True

In [52]:
print("Nemours" in tokenizer.get_vocab())
print(is_also_proper("Cnx Resources","Alpha Natural Resources Anr",nlp))
print(is_also_proper("BNY Mellon","Bank of New York Mellon",nlp))
print(is_both_unique("Yum Brands","Conagra Brands"))
print(is_both_unique("Sl Green Realty","Slg"))
print(is_both_unique("Dupont De Nemours","Ei Du Pont De Nemours"))
print(is_valid_match("Jp Morgan Chase","Jpmorgan"))
print(is_valid_match("Bank America","Bank"))
print(difflib.SequenceMatcher(None,"Sl Green Realty","Green Realty").ratio())
print(difflib.SequenceMatcher(None,"Dupont De Nemours","Ei Du Pont De Nemours").ratio())

False
True
True
False
False
True
False
False
0.8888888888888888
0.8421052631578947


In [46]:
#matching:
res_df = pd.DataFrame(columns=['Real company','Extracted company','Metrix','Comment','Article'])

found_num = 0
not_found_num = 0
total = 0

labeled_companies = df_labels['Name']
news = df_news['News']

COEFFS = {'contain_real': 0.9,'is_valid_match': 0.8,'is_both_unique': 0.8,'is_also_proper': 0.8,'is_abbreviation': 0.5,'is_also_person':0.8}
ARTICLES_NUM = len(labeled_companies)
extracted_companies = df_all_extracted_companies['Name']
perfect_companies = 0
contain_real_companies = 0

for idx in range (ARTICLES_NUM):
    labeled = labeled_companies[idx].split('\t')
    extracted = extracted_companies[idx].split('\t')
    total += len(labeled)
    for labeled_elem in labeled:
        max_metrix = 0.0
        COMMENT = ""
        for extracted_elem in extracted:
            labeled_str = labeled_elem.replace(" ","").replace("-","").lower()
            extracted_str = extracted_elem.replace(" ","").replace("-","").lower()
            metrix = difflib.SequenceMatcher(None,labeled_str,extracted_str).ratio()
                
            if (metrix == 1.0):
                max_elem = extracted_elem
                max_metrix = 1.0
                COMMENT = ""
                perfect_companies += 1
                break
            if (is_extracted_contain_real(labeled_elem,extracted_elem)):
                max_elem = extracted_elem
                max_metrix = 1.0
                contain_real_companies += 1
                COMMENT = "Extracted company contain real"
                break
            if (metrix > 0.3 and len(extracted_str) > 2 and (len(labeled_str) > 2) and (is_valid_match(labeled_elem,extracted_elem) or is_valid_match(extracted_elem,labeled_elem) or is_also_proper(labeled_elem,extracted_elem,nlp) or is_both_unique(labeled_elem,extracted_elem))):
                max_metrix = 0.85
                max_elem = extracted_elem
            if metrix > max_metrix:
                max_metrix = metrix
                max_elem = extracted_elem
                COMMENT = ""
            if (is_abbreviation(labeled_elem,extracted_elem) or is_abbreviation(extracted_elem,labeled_elem)):
                max_metrix = 0.81
                max_elem = extracted_elem
        if (max_metrix > 0.95)or(max_metrix > 0.8  and len(labeled_elem) > 2 and len(max_elem) > 2):      
            res_df.loc[found_num]=[labeled_elem,max_elem,max_metrix,COMMENT,news[idx]]
            found_num += 1
        else:
            not_found_num += 1
            #print(labeled_elem,"+", extracted," best: ",max_elem," :idx = ",idx)
print("not found", not_found_num)
print("found", found_num)
print("total",total)
print("found_res = ",found_num/total)
print("perfect_res = ",perfect_companies/total)
print("contained_res = ",contain_real_companies/total)
print(res_df.count())
res_df.drop_duplicates(subset=['Real company','Extracted company'],inplace=True)
print(res_df.count())
res_df.to_csv("results/bert_sttict_match.csv",index=False)


Sentinelone + ['Sprint Nextel', 'Clearwire', 'Att', 'Sprint Nextel', 'Wall Street Journal', 'Sprint', 'Journal', 'Clearwire', 'Sprint', 'Sprint', 'Att', 'Att', 'Sprint']  best:  Sprint Nextel  :idx =  12
Visa + ['Capital One Financial', 'Jpmorgan Chase', 'Bank America', 'Bank America', 'Bank America', 'Citigroup', 'Citigroup', 'Citigroup', 'American Express', 'American Express', 'American Express', 'Capital One Financial', 'Co', 'Capital One', 'Capital One', 'Capital One', 'Jpmorgan Chase', 'Jpm', 'Bank America', 'Bac', 'Citigroup', 'American Express', 'Disco', 'Dfs', 'Capital One', 'Masterc', 'Ma', 'Capital One', 'New York Stock Exchange']  best:  Disco  :idx =  21
Citigroup + ['Globe And Mail-Bp', 'Business Section-Shoppers Drug Mart', 'Xerox', 'Reuters', 'Globe', 'Bp', 'Bp', 'Shoppers', 'Drug Mart', 'Shoppers Op', 'Xerox', 'Xerox Canada', 'Canada Pension Plan', 'Rcmp', 'Imperial Bank', 'Commerce', 'Master']  best:  Shoppers Op  :idx =  23
Callaway Golf + ['Pfizer', 'Trubion', 'Lin',

Citigroup + ['London-Based Hsbc', 'Hsbc', 'Senate', 'Hsbc', 'Transparency International', 'Hsbc Bank USa', 'Hsbc Bank USa', 'Hsbc', 'Hsbc', 'US Senate', 'Hsbc', 'Hsbc', 'Do']  best:  Transparency International  :idx =  469
Grupo Aeroportuario Del Sureste Sa De Cv + ['Not Avaliable', 'Asur', 'Copenhagen Airports', 'Asur', 'Copenhagen Airports', 'Inversiones Tecnicas Aeroportuarias', 'Ita', 'Asur', 'Pardo', 'Ita', 'Asur', 'Asur', 'Reuters', 'Asur', 'Asur', 'Copenhagen Airports', 'Ita', 'Asur', 'Asur', 'Pardo', 'Ita', 'Asur Bb', 'Asur', 'Cy', 'Barr', 'Gunn']  best:  Inversiones Tecnicas Aeroportuarias  :idx =  481
Grupo Aeroportuario Del Sureste Sa De Cv + ['Not Avaliable', 'Asur', 'Copenhagen Airports', 'Asur', 'Copenhagen Airports', 'Inversiones Tecnicas Aeroportuarias', 'Ita', 'Asur', 'Pardo', 'Ita', 'Asur', 'Asur', 'Reuters', 'Asur', 'Asur', 'Copenhagen Airports', 'Ita', 'Asur', 'Asur', 'Pardo', 'Ita', 'Asur Bb', 'Asur', 'Cy', 'Barr', 'Gunn']  best:  Inversiones Tecnicas Aeroportuaria

Citigroup + ['Temasek', 'Dbs', 'Dbsm', 'Si', 'Temasek', 'Tem', 'Ul', 'United Oveseas Bank', 'Oversea', 'Chinese Banking', 'Ocbc', 'Si', 'Association Banks In', 'Standard Chartered']  best:  Association Banks In  :idx =  744
Citigroup + ['Temasek', 'Dbs', 'Dbsm', 'Si', 'Temasek', 'Tem', 'Ul', 'United Oveseas Bank', 'Oversea', 'Chinese Banking', 'Ocbc', 'Si', 'Association Banks In', 'Standard Chartered']  best:  Association Banks In  :idx =  744
Lazard + ['Not Avaliable', 'Le Monde', 'Laz', 'Reuters', 'Le Monde', 'Iliad', 'Ild Pa', 'Nouvel Observateur', 'France Telecom Fte Pa', 'Prisa', 'Prs Mc', 'El Pais', 'Le Monde']  best:  Laz  :idx =  749
Kb Home + ['Jpmorgan Chase', 'Jpmorgan Chase', 'Motion', 'Apple', 'Oracle', 'Congress', 'Id', 'Jpmorgan Chase', 'Jpm', 'Bkx', 'Solaris', 'Dji', 'Spx', 'Ixic', 'Sp', 'Research', 'Ltd', 'Apple', 'Id']  best:  Oracle  :idx =  751
Caleres + ['Ual', 'United Airlines', 'Ual', 'Continental Airlines', 'Cal', 'Air Line Pilots Association', 'Alpa']  best:  C

Dupont De Nemours + ['Not Avaliable', 'European Union', 'European Commission', 'Commission', 'Eu', 'Reuters', 'Eu Health And Consumer', 'Eu', 'Commission', 'Eu', 'Eu', 'Syngenta Synn', 'Vx', 'Eu', 'Syngenta', 'Syngenta', 'Du', 'Du', 'Anthony']  best:  Reuters  :idx =  933
Dow + ['Not Avaliable', 'European Union', 'European Commission', 'Commission', 'Eu', 'Reuters', 'Eu Health And Consumer', 'Eu', 'Commission', 'Eu', 'Eu', 'Syngenta Synn', 'Vx', 'Eu', 'Syngenta', 'Syngenta', 'Du', 'Du', 'Anthony']  best:  Du  :idx =  933
Petrochina + ['Not Avaliable', 'Bkadr', 'Bkeur', 'Barclays', 'Bcs', 'Credit Suisse', 'Cs', 'Ubs', 'Ubs', 'Bkas', 'Conference Board', 'Bell', 'Toyota']  best:  Toyota  :idx =  940
Caleres + ['Continental Airlines', 'United Airlines', 'Uaua', 'Continental Airlines', 'Cal', 'District Court', 'United', 'Continental', 'United', 'Ual', 'Continental', 'United', 'Justice Department', 'Continental', 'Continental', 'United', 'Doj', 'Malaney', 'Ual', 'District Court Northern Dist

Glaxosmithkline + ['Par Pharmaceutical Companies', 'Par Pharmaceutical Companies', 'Gsk', 'Gsk', 'Zofran', 'Strativa Pharmaceuticals', 'Par', 'Par', 'New York Stock Exchange']  best:  Gsk  :idx =  1194
Fomento Economico Mexicano + ['Not Avaliable', 'Kansas City Southern', 'Ksu', 'Fems', 'Fm', 'Fmx', 'Ce', 'Santa', 'Mohammad']  best:  Mohammad  :idx =  1199
Kroger + ['Not Avaliable', 'Agriculture Department', 'Rocky Mountain Natural Meats', 'USda', 'USda', 'USda', 'USda', 'Hannaford', 'King Soopers', 'Market Basket', 'Price Chopper', 'Whole Foods Wfm', 'USda', 'USda', 'Centers For Disease Control And Prevention', 'Rocky Mountain Natural Meats', 'Re']  best:  Re  :idx =  1205
Manpowergroup + ['Not Avaliable', 'Randstad', 'Rand As', 'Aex', 'Ubs', 'Theodoor Gilissen', 'Gilissen', 'Manpower', 'Man', 'Adecco Aden', 'Vx', 'Randstad', 'Gilissen', 'Reuters Me', 'Reuters', 'Re']  best:  Manpower  :idx =  1249
Boeing + ['Kuwait Aviation Lease And Finance', 'Kuwait Aviation Lease And Finance', 'Al

...

Citigroup + ['Jpmorgan Chase', 'Goldman Sachs Group', 'Goldman Sachs Group', 'Jpmorgan Chase', 'Jpm', 'Jpmorgan', 'Bloomberg', 'Goldman Sachs Group', 'Gs']  best:  Goldman Sachs Group  :idx =  53105
Citigroup + ['Jpmorgan Chase', 'Goldman Sachs Group', 'Goldman Sachs Group', 'Jpmorgan Chase', 'Jpm', 'Jpmorgan', 'Bloomberg', 'Goldman Sachs Group', 'Gs']  best:  Goldman Sachs Group  :idx =  53106
General Electric + ['Embraer Sa', 'Boeing', 'Embraer Sa', 'Sa', 'Boeing', 'Ba', 'Embraer', 'Ge Aviation', 'Ge', 'Boeing', 'Embraer', 'Boeing', 'Embraer', 'Boeing', 'Embraer', 'Embraer', 'Embraer', 'Embraer', 'Bndes', 'Embraer', 'Bndes', 'Embraer', 'Em', 'Em', 'Em', 'Mei', 'Em', 'Em']  best:  Ge  :idx =  53117
General Electric + ['Embraer Sa', 'Boeing', 'Embraer Sa', 'Sa', 'Boeing', 'Ba', 'Embraer', 'Ge Aviation', 'Ge', 'Boeing', 'Embraer', 'Boeing', 'Embraer', 'Boeing', 'Embraer', 'Embraer', 'Embraer', 'Embraer', 'Bndes', 'Embraer', 'Bndes', 'Embraer', 'Em', 'Em', 'Em', 'Mei', 'Em', 'Em']  best:

Chevron + ['Hibernia Management And Development', 'Suncor Energy', 'Suncor Energy', 'Hibernia Management And Development', 'Hmdc', 'Hibernia', 'Exxonmobil', 'Ch', 'Suncor Energy', 'Su', 'To', 'Newfoundland And Labrador Offshore Petroleum Board']  best:  Hibernia  :idx =  53704
Citigroup + ['American Express', 'Susquehanna Financial Group', 'Jpmorgan Chase', 'American Express', 'Amex', 'Susquehanna Financial Group', 'Reuters', 'Jpmorgan Chase', 'Jpm', 'American Express', 'New York Stock Exchange', 'Nyse', 'Reutersbren', 'American Express', 'Am']  best:  Susquehanna Financial Group  :idx =  53766
Citigroup + ['American Express', 'Susquehanna Financial Group', 'Jpmorgan Chase', 'American Express', 'Amex', 'Susquehanna Financial Group', 'Reuters', 'Jpmorgan Chase', 'Jpm', 'American Express', 'New York Stock Exchange', 'Nyse', 'Reutersbren', 'American Express', 'Am']  best:  Susquehanna Financial Group  :idx =  53767
Jefferies Financial + ['Glaxosmithkline', 'Microsoft', 'Curevac', 'Curevac

Jp Morgan Chase + ['New York Stock Exchange And Nasdaq', 'Nyse-Owner Intercontinental Exchange', 'Iex Group', 'Blackrock', 'Charles Schwab', 'New York Stock Exchange', 'Nasdaq', 'Ndaq', 'Long Term Stock Exchange', 'Ltse', 'Members Exchange', 'Memx', 'Miax Pearl Equities', 'Intercontinental Exchange', 'Cboe Global Markets', 'Cboe', 'Nasdaq', 'Iex Group', 'Memx', 'Memx', 'Jpmorgan', 'Jpm', 'Bank America', 'Bac', 'Goldman Sachs', 'Gs', 'Blackrock', 'Charles Schwab', 'Schw', 'Td', 'Am', 'Citadel', 'Securities', 'Mi']  best:  Jpmorgan  :idx =  54160
Far Peak Acquisition + ['Partners Group', 'Silver Lake And Partners Group', 'Far Point', 'Fpac', 'Third Point Llc', 'Global Blue', 'Third Point', 'Global Blue', 'Far Point', 'Far Point', 'Far Point', 'Global Blue', 'Far Point', 'Silver Lake', 'Global Blue', 'Partners Group Holding Ag', 'Far Point', 'Silver Lake', 'Partners Group', 'Global Blue', 'Far Point', 'Spac', 'Ip', 'Global Blue', 'New York Stock Exchange']  best:  Far Point  :idx =  54180

Conocophillips + ['Not Avaliable', 'Reuters', 'Brent', 'Norwegian Oil And Gas Association', 'Nog', 'Lederne', 'Akerbp', 'Ol', 'Equinor', 'Ol', 'Nog', 'Lederne', 'Nog']  best:  Not Avaliable  :idx =  54935
Conocophillips + ['Not Avaliable', 'Reuters', 'Brent', 'Norwegian Oil And Gas Association', 'Nog', 'Lederne', 'Akerbp', 'Ol', 'Equinor', 'Ol', 'Nog', 'Lederne', 'Nog']  best:  Not Avaliable  :idx =  54936
Conocophillips + ['Not Avaliable', 'Reuters', 'Brent', 'Norwegian Oil And Gas Association', 'Nog', 'Lederne', 'Akerbp', 'Ol', 'Equinor', 'Ol', 'Nog', 'Lederne', 'Nog']  best:  Not Avaliable  :idx =  54937
Citigroup + ['Not Avaliable', 'Siemens', 'Siegn', 'De', 'Triton', 'Flender', 'Siemens', 'Triton', 'Carlyle', 'Cg', 'Cvc', 'Brookfield', 'Siemens Energy', 'De', 'Healthineers', 'Shlg', 'De', 'Siemens', 'Citi', 'Bank America', 'Bac', 'Flender', 'Winergy', 'Triton', 'Renk', 'Zarg', 'Flender', 'Renk', 'Siemens', 'Gamesa', 'Sgren', 'Mc', 'Siemens', 'Siemens', 'Bo', 'Siemens']  best:  Cg 

Rex American Resources + ['Not Avaliable', 'Abb', 'Abbn', 'Dodge', 'Reuters', 'Dodge', 'Abb', 'Dodge', 'Abb', 'Timken', 'Regal Beloit', 'Rbc', 'Rexnord', 'Rex', 'Schaeffler', 'Sha', 'De', 'Sks', 'Skf', 'St', 'Dodge', 'Reuters', 'Abb', 'Abb', 'Vrt', 'Abb', 'Rosen', 'Reuters', 'War', 'Wr', 'Alfa', 'La', 'Alfa', 'St', 'Abb', 'Tim', 'Reg', 'Bel', 'Rex', 'Sc']  best:  Reuters  :idx =  55701
Vertiv + ['Not Avaliable', 'Abb', 'Abbn', 'Dodge', 'Reuters', 'Dodge', 'Abb', 'Dodge', 'Abb', 'Timken', 'Regal Beloit', 'Rbc', 'Rexnord', 'Rex', 'Schaeffler', 'Sha', 'De', 'Sks', 'Skf', 'St', 'Dodge', 'Reuters', 'Abb', 'Abb', 'Vrt', 'Abb', 'Rosen', 'Reuters', 'War', 'Wr', 'Alfa', 'La', 'Alfa', 'St', 'Abb', 'Tim', 'Reg', 'Bel', 'Rex', 'Sc']  best:  Vrt  :idx =  55701
Dupont De Nemours + ['Corteva', 'Corteva', 'Ctva', 'Starboard Value Lp', 'Corteva', 'Dowdupont', 'Dd', 'Starboard', 'Corteva', 'Corteva', 'Corteva', 'Starboard', 'Starboard', 'Corteva', 'Agriculture Division', 'Dowdupont', 'Corteva', 'Starboa

In [47]:
# USING conll2003 pretrained
#result - 59% for pretrained 1000 companies
# 58,5% for 10000 companies

#after handling
#63.5% for oretrained 1000 
#

#after ramoving long articles - 
#0.792 for 1000 articles
#0.877 for all articles
#afrer removing "&" - 0.83295
#holdngs and group reamoval: - 0.8382
#0.81 - spacy, but probably more false positive

In [48]:
#only nasdaq:
#93,7% - bert ->94,18%
#90.9% - spacy
#52.3% - nltk
#95% - flair

In [49]:
#add  to normalization:
#if(elem.find(" ")!=-1 and extracted_elem.find(" ")!=-1):# delete words except first with len < 3 if both companies have more than 1 word
#                labeled_elem = re.sub(r' \w{1,2}\b', '', labeled_elem).strip()
#                extracted_elem = re.sub(r' \w{1,2}\b', '', extracted_elem).strip()

In [50]:
#94.7% (73,6% perfect)- spacy
#95,7% (73,4% perfect) - bert
#0.95 coeff for validmatch and other 
#found_res =  0.9570530076540422
#perfect_res =  0.7429259034126856
#contained_res =  0.10295121013261217
#0.85 coeff for validmatch
#
#no the
#found_res =  0.9525623053925643
#perfect_res =  0.7489052362580789
#contained_res =  0.1019215739787374
#strict
#found_res =  0.945615362667626
#perfect_res =  0.7489052362580789
#contained_res =  0.1019215739787374